In [14]:
import numpy as np
import pandas as pd

import time
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go


# 1. Data Preparation

3 datasets:

S&P 500 daily adjusted price

Mean-variance weights table

Min-variance weights table

In [15]:
data_adj = pd.read_csv("final_ratios_oct.csv")
weight_meanv = pd.read_excel('mean_weighted_rf.xlsx')
weight_minv = pd.read_excel('minimum_weighted_rf.xlsx')
weight_equal=pd.read_excel('equally_weighted_rf.xlsx')

In [16]:
data_adj

,Unnamed: 0,date,gvkey,tic,gsector,adj_close_q,y_return,EPS,DPS,cur_ratio,quick_ratio,cash_ratio,acc_rec_turnover,debt_ratio,debt_to_equity,pe,ps,pb
0,66261,1998-03-01,126554,A,35.0,0.00,0.000000,0.42,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,66262,1998-06-01,126554,A,35.0,0.00,0.000000,0.67,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,66263,1998-09-01,126554,A,35.0,0.00,0.000000,0.81,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,66264,1998-12-01,126554,A,35.0,0.00,0.000000,0.68,0.000,1.923077,0.759850,0.000000,4.909465,0.394024,0.000000,0.000000,0.000000,0.000000
4,66265,1999-03-01,126554,A,35.0,0.00,0.000000,0.19,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77580,41126,2021-09-01,13721,ZTS,35.0,186.36,0.040899,2.25,0.250,3.370636,2.308943,1.749402,4.676923,0.690667,2.232767,172.555556,45.319980,20.306517
77581,41127,2021-12-01,13721,ZTS,35.0,194.14,0.228712,3.42,0.250,4.542488,3.008838,2.225697,4.889757,0.658446,1.927793,167.362069,46.111805,19.632734
77582,41128,2022-03-01,13721,ZTS,35.0,244.03,-0.257716,4.29,0.250,3.856427,2.569839,1.939343,5.133274,0.673094,2.058979,277.306818,58.539205,25.379013
77583,41129,2022-06-01,13721,ZTS,35.0,188.59,-0.092721,1.26,0.325,2.253461,1.436058,1.033289,4.838789,0.663925,1.975526,149.674603,44.723354,19.077996


In [17]:
weight_meanv.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2001-03-01,1078,0.00000,0.044809
1,1,2001-03-01,1230,0.00000,0.019396
2,2,2001-03-01,1246,0.01237,0.014087
3,3,2001-03-01,1327,0.01888,0.128518
4,4,2001-03-01,1602,0.00000,0.045424


In [18]:
weight_minv.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2001-03-01,1078,0.00662,0.044809
1,1,2001-03-01,1230,0.00631,0.019396
2,2,2001-03-01,1246,0.00629,0.014087
3,3,2001-03-01,1327,0.00628,0.128518
4,4,2001-03-01,1602,0.00628,0.045424


In [19]:
weight_equal.head()

,Unnamed: 0,trade_date,gvkey,weights,predicted_return
0,0,2001-03-01,1078,0.006579,0.044809
1,1,2001-03-01,1230,0.006579,0.019396
2,2,2001-03-01,1246,0.006579,0.014087
3,3,2001-03-01,1327,0.006579,0.128518
4,4,2001-03-01,1602,0.006579,0.045424


In [20]:
#the length of the 3 tables should be the same
print(len(weight_meanv),
len(weight_minv),
len(weight_equal))


16319 16319 16319


## 1.1 Get stock name, trade date, daily date

Add 20170901 as tradedate_full


In [21]:
#get unique stock name from table
#stock names are the same between mean variance and minimum variance
#only the weights are different
stocks_name = list(set(weight_meanv['gvkey']))
stocks_name.sort()

In [22]:
len(stocks_name)

858

In [23]:
#get all daily date
#convert to list from pandas.core.series.Series
#use set to get unique values
daily = list(set(data_adj['date']))
#sort it
daily.sort()
print(type(data_adj['date']),type(daily))

<class 'pandas.core.series.Series'> <class 'list'>


In [24]:
len(daily)


108

In [25]:
#get unique trade date from table
tradedate = list(set(weight_meanv['trade_date']))
tradedate.sort()


In [26]:
tradedate

['2001-03-01',
 '2001-06-01',
 '2001-09-01',
 '2001-12-01',
 '2002-03-01',
 '2002-06-01',
 '2002-09-01',
 '2002-12-01',
 '2003-03-01',
 '2003-06-01',
 '2003-09-01',
 '2003-12-01',
 '2004-03-01',
 '2004-06-01',
 '2004-09-01',
 '2004-12-01',
 '2005-03-01',
 '2005-06-01',
 '2005-09-01',
 '2005-12-01',
 '2006-03-01',
 '2006-06-01',
 '2006-09-01',
 '2006-12-01',
 '2007-03-01',
 '2007-06-01',
 '2007-09-01',
 '2007-12-01',
 '2008-03-01',
 '2008-06-01',
 '2008-09-01',
 '2008-12-01',
 '2009-03-01',
 '2009-06-01',
 '2009-09-01',
 '2009-12-01',
 '2010-03-01',
 '2010-06-01',
 '2010-09-01',
 '2010-12-01',
 '2011-03-01',
 '2011-06-01',
 '2011-09-01',
 '2011-12-01',
 '2012-03-01',
 '2012-06-01',
 '2012-09-01',
 '2012-12-01',
 '2013-03-01',
 '2013-06-01',
 '2013-09-01',
 '2013-12-01',
 '2014-03-01',
 '2014-06-01',
 '2014-09-01',
 '2014-12-01',
 '2015-03-01',
 '2015-06-01',
 '2015-09-01',
 '2015-12-01',
 '2016-03-01',
 '2016-06-01',
 '2016-09-01',
 '2016-12-01',
 '2017-03-01',
 '2017-06-01',
 '2017-09-

In [27]:
tradedate_full = [x for x in tradedate if x > '2018-01-01']

In [28]:
tradedate_full

['2018-03-01',
 '2018-06-01',
 '2018-09-01',
 '2018-12-01',
 '2019-03-01',
 '2019-06-01',
 '2019-09-01',
 '2019-12-01',
 '2020-03-01',
 '2020-06-01',
 '2020-09-01',
 '2020-12-01',
 '2021-03-01',
 '2021-06-01',
 '2021-09-01',
 '2021-12-01',
 '2022-03-01',
 '2022-06-01',
 '2022-09-01',
 '2022-12-01']

## 1.2 Get the daily price matrix based on stock_name

In [29]:
#30mins running time
##################match current all stocks to daily adjusted price table#######################
# backtest daily price matrix:
# covert original table to matrix
# rows = all stocks: 971, columns = all dates:7155 days
#
start_time = time.time()

balance_daily = pd.DataFrame(index = stocks_name, columns = daily)
balance_daily = balance_daily.fillna(np.nan)

for i in stocks_name:
    ind = np.where(data_adj['gvkey'] == i)[0]
    tmp = data_adj.loc[ind, :]
    tmp.index = range(tmp.shape[0])
    for j in daily:
        ind2 = np.where(tmp['date'] == j)[0]
        if(len(ind2) != 0):
            balance_daily.loc[i,j] = tmp.loc[ind2[0], 'adj_close_q']

elapsed_time = time.time() - start_time
print(elapsed_time)
writer2 = pd.ExcelWriter('balance_daily.xlsx')
writer2 = pd.ExcelWriter('balance_daily_user8.xlsx')

balance_daily.to_excel(writer2,'Sheet1')
writer2.save()
###############################################

14.190996885299683


In [30]:
balance_daily = pd.read_excel('balance_daily_user8.xlsx', index_col=0)


In [31]:
balance_daily

,1996-03-01,1996-06-01,1996-09-01,1996-12-01,1997-03-01,1997-06-01,1997-09-01,1997-12-01,1998-03-01,1998-06-01,...,2020-09-01,2020-12-01,2021-03-01,2021-06-01,2021-09-01,2021-12-01,2022-03-01,2022-06-01,2022-09-01,2022-12-01
1045,NaN,44.7500,45.5000,39.81250,44.0625,41.25000,46.25000,55.34350,64.2500,71.59350,...,13.07,12.29,15.77,23.90,21.21,20.52,17.96,18.25,12.68,12.04
1075,NaN,28.8750,30.3750,29.62500,31.7500,30.12500,30.06200,33.62500,42.3750,44.43700,...,73.29,74.55,79.95,81.35,81.97,72.36,70.59,78.10,73.12,NaN
1078,NaN,20.3750,21.7500,24.62500,25.3750,28.06250,33.37500,31.96850,32.7500,37.65600,...,91.43,108.83,109.49,119.84,115.93,118.13,140.74,118.36,108.65,96.76
1161,NaN,8.6875,6.8125,7.37500,12.8750,20.75000,17.93750,16.28100,8.8750,14.53100,...,52.61,81.99,91.71,78.50,93.93,102.90,143.90,109.34,76.47,NaN
1177,NaN,18.8750,17.8750,17.59375,20.0000,21.46875,25.59375,20.35925,17.6405,20.85925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.72,1.51,2.05,1.73,2.03,4.06,3.18,NaN,NaN,NaN
287882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.13,9.84,12.91,12.82,14.62,14.35,12.89,18.60,10.86,NaN
294524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.72,70.49,91.66,104.05,102.87,93.85,92.23,102.82,87.46,NaN
312009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.98,6.50,6.22,7.15,9.00,9.37,7.07,8.80,8.86,NaN


# 2. Function: calculate the rebalance matrix:

1. shares table
2. cost table
3. cash table
4. portfolio table
5. portfolio_cost table
6. portfolio_return table
7. portfolio_cumsum table

In [32]:
# set initial capital to 1 million and transaction percet to 0.1%

capital = 1000000
transaction_percent = 1/1000

In [33]:
weight_meanv.shape

(16319, 5)

In [38]:
#construct an empty matrix
#rows are stocks: 971 unique stocks
#columns are trade dates
#name the index as stock names

def cal_portfolio(stocks_name,tradedate, weight_table, capital, transaction_percent):


    balance_weight = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_weight = balance_weight.fillna(0)
    
    #loop through the weights table to fill the matrix
    #need weight tabele
    #0. weights table
    for i in range(weight_table.shape[0]):
        tics = weight_table.loc[i, 'gvkey']
        date = weight_table.loc[i, 'trade_date']
        w = weight_table.loc[i, 'weights']
        balance_weight.loc[tics, date] = w
        
    #1. shares table
    balance_share = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_share['initial'] = 0
    balance_share = balance_share.fillna(0)
    #2. costs table
    balance_cost = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_cost = balance_cost.fillna(0)
    #3. cash table
    balance_cash = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_cash = balance_cash.fillna(0)

    #4. portfolio without transaction costs table 
    portfolio = pd.DataFrame(index = range(1), columns = tradedate_full)
    portfolio.loc[0,tradedate[0]] = capital
    
    
    # calculate portfolio value without cost
    balance_price_full = []
    balance_price_full = balance_daily[tradedate_full]
    balance_price = balance_daily[tradedate]
    balance_price.replace([0, np.inf, -np.inf], np.nan, inplace=True)
    for j in range(len(tradedate) - 1):
        i = tradedate[j]
        #current capital
        cap = portfolio.loc[0, i]
        #
        balance_cash[i] = cap * balance_weight[i]
        balance_share[i] = list(np.array(balance_cash[i])
                                      / np.array(balance_price[i]))
        balance_share[i] = balance_share[i].fillna(0)
        balance_share[i] = balance_share[i].astype(int)
        #next quarter price * current share to calculate next quarter total account value
        a = balance_price_full[tradedate_full[j+1]].fillna(0)
        portfolio.loc[0, tradedate_full[j+1]] = np.dot(balance_share[i], a)

    balance_cost = abs(np.diff(balance_share, axis = 1))
    balance_cost = balance_cost * balance_price
    balance_cost = np.sum(balance_cost, axis = 0) * transaction_percent
    
    #5. portfolio with transaction cost table
    #calcualte portolio value with transaction cost
    balance_cost.index = tradedate_full
    portfolio_cost = []
    portfolio_cost = portfolio.loc[0,] - balance_cost
    portfolio_cost.index = tradedate_full
    portfolio_cost_value = portfolio_cost.copy()
    
    #6. portfolio return P&L table
    portfolio_cumsum = portfolio_cost_value.copy()
    portfolio_cumsum.index = range(len(portfolio_cumsum))
    tmp = portfolio_cumsum.shift(1)
    tmp[0] = 0
    tmp.index = tradedate_full

    portfolio_cumsum = portfolio_cost_value- tmp
    portfolio_cumsum.index = range(len(portfolio_cumsum))
    tmp.index = range(len(tmp))
    for i in range(len(portfolio_cumsum)):
        if(tmp[i] != 0):
            portfolio_cumsum[i] = portfolio_cumsum[i]/tmp[i]

    portfolio_cumsum.index = range(len(portfolio_cumsum))
    portfolio_cumsum[0] = 0
    portfolio_return=portfolio_cumsum.copy()



    # 7. portfolio accumulative return table
    initial = 0
    for i in range(portfolio_cumsum.shape[0]):
        portfolio_cumsum[i] = initial + portfolio_cumsum[i]
        initial = portfolio_cumsum[i]
    
    #1. shares table
    #2. costs table
    #3. cash table
    #4. portfolio without transaction costs table 
    #5. portfolio with transaction cost table
    #6. portfolio return P&L table
    #7. portfolio accumulative return table

    return(balance_share,
           balance_cost,
           balance_cash,
           portfolio,
           portfolio_cost,
           portfolio_return,
           portfolio_cumsum)


In [39]:
#calculate mean variance allocation portolio
(balance_share_meanv,
 balance_cost_meanv,
 balance_cash_meanv,
 portfolio_meanv,
 portfolio_cost_meanv,
 portfolio_return_meanv,
 portfolio_cumsum_meanv)=cal_portfolio(stocks_name,tradedate_full, weight_meanv, capital, transaction_percent)


/tmp/ipykernel_3642/1845856610.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_price.replace([0, np.inf, -np.inf], np.nan, inplace=True)


In [40]:
sum(portfolio_return_meanv)

-0.6256100382905669

In [41]:
#calculate minimum variance allocation portolio
(balance_share_minv,
 balance_cost_minv,
 balance_cash_minv,
 portfolio_minv,
 portfolio_cost_minv,
 portfolio_return_minv,
 portfolio_cumsum_minv)=cal_portfolio(stocks_name,tradedate_full, weight_minv, capital, transaction_percent)

/tmp/ipykernel_3642/1845856610.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_price.replace([0, np.inf, -np.inf], np.nan, inplace=True)


In [42]:
#calculate equally weighted allocation portolio
(balance_share_equal,
 balance_cost_equal,
 balance_cash_equal,
 portfolio_equal,
 portfolio_cost_equal,
 portfolio_return_equal,
 portfolio_cumsum_equal)=cal_portfolio(stocks_name,tradedate_full, weight_equal, capital, transaction_percent)

/tmp/ipykernel_3642/1845856610.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_price.replace([0, np.inf, -np.inf], np.nan, inplace=True)


In [43]:
sum(portfolio_return_equal)

-0.6039422883904094

# 3. Calculate SPX Portfolio Value, Return and Cumulative Return

In [54]:
import datetime

dates = pd.date_range('1999-09-01', '2022-09-01')


In [55]:
spx = pd.read_csv("SPX.csv")


In [56]:
spx.date = pd.to_datetime(spx.date)

In [57]:
spx = spx.set_index("date").reindex(dates, method='ffill')

In [ ]:
spx

0     2677.669922
1     2734.620117
2     2901.520020
3     2760.169922
4     2803.689941
5     2752.060059
6     2926.459961
7     3140.979980
8     2954.219971
9     3055.729980
10    3526.649902
11    3662.449951
12    3901.820068
13    4202.040039
14    4524.089844
15    4513.040039
16    4306.259766
Name: close, dtype: float64

In [58]:
tradedate = tradedate[:-1]

In [59]:

spx_daily = list(set(spx.index))
spx_daily.sort()
spx = spx['close']
spx.index = spx.index.astype(str)
spx = spx[tradedate]
spx.index = range(len(spx))
spx_return = (spx - spx.shift(1))/spx
spx_return[0] = 0

spx_return.index = tradedate
spx_return.index = range(len(spx))
spx_cumsum=spx_return.copy()

spx_value = spx_return.copy()
initial = 0
for i in range(len(spx_cumsum)):
    spx_cumsum[i] = initial + spx_cumsum[i]
    initial = spx_cumsum[i]

initial = 1000000
for i in range(len(spx_value)):
    spx_value[i] = initial * (1 + spx_value[i])
    initial = spx_value[i]

spx_cumsum.index = tradedate
spx_value.index = tradedate

In [60]:
sum(spx_return)

0.8455213218848242

# 4. Plot the Portfolio Performance

In [ ]:
time_ind.head()

NameError: name 'time_ind' is not defined

## 4.1 Plot Portfolio Value with Transaction Cost

In [65]:
time_ind = tradedate_full[:-1]

In [66]:
trace0 = go.Scatter(x = time_ind, y = portfolio_cost_meanv, mode = 'lines', name = 'Mean-Variance')
trace1 = go.Scatter(x = time_ind, y = portfolio_cost_equal, mode = 'lines', name = 'Equally Weighted')
trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

data = [trace0, trace1, trace2, trace3]
layout = dict(title = "Portfolio Value Performance Quarterly with Transaction cost", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)

plotly.offline.iplot(fig)

#portfolio_tmp = portfolio_cost_value

## 4.2 Plot Portfolio Return

In [67]:
trace_return1 = go.Scatter(x = time_ind, y = portfolio_return_meanv, mode = 'lines', name = 'Mean-Variance')
trace_return2 = go.Scatter(x = time_ind, y = portfolio_return_minv, mode = 'lines', name = 'Minimum-Variance')
trace_return3 = go.Scatter(x = time_ind, y = portfolio_return_equal, mode = 'lines', name = 'Equally Weighted')
trace_return4 = go.Scatter(x = time_ind, y = spx_return, mode = 'lines', name = 'SPX')

data = [trace_return1,trace_return2,trace_return3, trace_return4]
layout = dict(title = "Portfolio Return Quarterly", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)
plotly.offline.iplot(fig)

## 4.3 Plot Portfolio Cumulative Return

In [68]:
trace_return1 = go.Scatter(x = time_ind, y = portfolio_cumsum_meanv, mode = 'lines', name = 'Mean-Variance')
trace_return2 = go.Scatter(x = time_ind, y = portfolio_cumsum_minv, mode = 'lines', name = 'Minimum-Variance')
trace_return3 = go.Scatter(x = time_ind, y = portfolio_cumsum_equal, mode = 'lines', name = 'Equally Weighted')
trace_return4 = go.Scatter(x = time_ind, y = spx_cumsum, mode = 'lines', name = 'SPX')

data = [trace_return1,trace_return2,trace_return3, trace_return4]
layout = dict(title = "Portfolio Cumulative Return Quarterly", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)
plotly.offline.iplot(fig)

# 5. Get all Quarterly Return and Calcualte Annulized return, standard deviation and Sharpe ratio

In [69]:
quarter_return = pd.DataFrame(index = range(len(portfolio_return_equal)), columns = ['Mean-Var', 'Equally', 'Min-Var'])
quarter_return['SPX'] = spx_return
quarter_return['Mean-Var'] = portfolio_return_meanv
quarter_return['Equally'] = portfolio_return_equal
quarter_return['Min-Var'] = portfolio_return_minv

#writer_return = pd.ExcelWriter('quarter_return_user8.xlsx')
quarter_return.index=tradedate_full
#quarter_return.to_excel(writer_return,'Sheet1')
#writer_return.save()

In [70]:
quarter_return.head()

,Mean-Var,Equally,Min-Var,SPX
2018-03-01,0,0,0,0.000000
2018-06-01,0.02529,-0.01857,-0.006528,0.015420
2018-09-01,-0.018328,0.018824,0.016548,-0.112114
2018-12-01,0.026316,0.026002,0.024862,0.005152
2019-03-01,-0.214458,-0.16628,-0.164245,-0.006777


In [71]:
annulizedR_spx=quarter_return['SPX'].mean()*4
annulizedR_meanv=quarter_return['Mean-Var'].mean()*4
annulizedR_equal=quarter_return['Equally'].mean()*4
annulizedR_minv=quarter_return['Min-Var'].mean()*4
annulizedS_spx=quarter_return['SPX'].std()*2
annulizedS_meanv=quarter_return['Mean-Var'].std()*2
annulizedS_equal=quarter_return['Equally'].std()*2
annulizedS_minv=quarter_return['Min-Var'].std()*2

In [72]:
print("SPX",annulizedR_spx,annulizedS_spx)
print("Mean-Var",annulizedR_meanv,annulizedS_meanv)
print("Equally",annulizedR_equal,annulizedS_equal)
print("Min-Var",annulizedR_minv,annulizedS_minv)


SPX -0.00522854196153727 0.13913436271090593
Mean-Var -0.12512200765811338 0.5267684626050293
Equally -0.12078845767808188 0.4858170592420743
Min-Var -0.12281867413974365 0.4877126592890122


In [75]:
risk_free=0.015

In [76]:
sharpe_spx=(annulizedR_spx-risk_free)/annulizedS_spx
sharpe_meanv=(annulizedR_meanv-risk_free)/annulizedS_meanv
sharpe_equal=(annulizedR_equal-risk_free)/annulizedS_equal
sharpe_minv=(annulizedR_minv-risk_free)/annulizedS_minv

In [77]:
print("SPX Sharpe",sharpe_spx)
print("Mean-Var Sharpe",sharpe_meanv)
print("Equally Sharpe",sharpe_equal)
print("Min-Var Sharpe",sharpe_minv)


SPX Sharpe -0.1453885407415006
Mean-Var Sharpe -0.2660030309429834
Equally Sharpe -0.27950533044254594
Min-Var Sharpe -0.2825817036216689


In [78]:
insample_date=tradedate

In [79]:
insample_date

['2001-03-01',
 '2001-06-01',
 '2001-09-01',
 '2001-12-01',
 '2002-03-01',
 '2002-06-01',
 '2002-09-01',
 '2002-12-01',
 '2003-03-01',
 '2003-06-01',
 '2003-09-01',
 '2003-12-01',
 '2004-03-01',
 '2004-06-01',
 '2004-09-01',
 '2004-12-01',
 '2005-03-01',
 '2005-06-01',
 '2005-09-01',
 '2005-12-01',
 '2006-03-01',
 '2006-06-01',
 '2006-09-01',
 '2006-12-01',
 '2007-03-01',
 '2007-06-01',
 '2007-09-01',
 '2007-12-01',
 '2008-03-01',
 '2008-06-01',
 '2008-09-01',
 '2008-12-01',
 '2009-03-01',
 '2009-06-01',
 '2009-09-01',
 '2009-12-01',
 '2010-03-01',
 '2010-06-01',
 '2010-09-01',
 '2010-12-01',
 '2011-03-01',
 '2011-06-01',
 '2011-09-01',
 '2011-12-01',
 '2012-03-01',
 '2012-06-01',
 '2012-09-01',
 '2012-12-01',
 '2013-03-01',
 '2013-06-01',
 '2013-09-01',
 '2013-12-01',
 '2014-03-01',
 '2014-06-01',
 '2014-09-01',
 '2014-12-01',
 '2015-03-01',
 '2015-06-01',
 '2015-09-01',
 '2015-12-01',
 '2016-03-01',
 '2016-06-01',
 '2016-09-01',
 '2016-12-01',
 '2017-03-01',
 '2017-06-01',
 '2017-09-

In [80]:
quarter_return['SPX'][insample_date]


KeyError: "['2001-03-01', '2001-06-01', '2001-09-01', '2001-12-01', '2002-03-01', '2002-06-01', '2002-09-01', '2002-12-01', '2003-03-01', '2003-06-01', '2003-09-01', '2003-12-01', '2004-03-01', '2004-06-01', '2004-09-01', '2004-12-01', '2005-03-01', '2005-06-01', '2005-09-01', '2005-12-01', '2006-03-01', '2006-06-01', '2006-09-01', '2006-12-01', '2007-03-01', '2007-06-01', '2007-09-01', '2007-12-01', '2008-03-01', '2008-06-01', '2008-09-01', '2008-12-01', '2009-03-01', '2009-06-01', '2009-09-01', '2009-12-01', '2010-03-01', '2010-06-01', '2010-09-01', '2010-12-01', '2011-03-01', '2011-06-01', '2011-09-01', '2011-12-01', '2012-03-01', '2012-06-01', '2012-09-01', '2012-12-01', '2013-03-01', '2013-06-01', '2013-09-01', '2013-12-01', '2014-03-01', '2014-06-01', '2014-09-01', '2014-12-01', '2015-03-01', '2015-06-01', '2015-09-01', '2015-12-01', '2016-03-01', '2016-06-01', '2016-09-01', '2016-12-01', '2017-03-01', '2017-06-01', '2017-09-01', '2017-12-01'] not in index"

In [ ]:
annulizedR_spx_insample=quarter_return['SPX'][insample_date].mean()*4
annulizedR_meanv_insample=quarter_return['Mean-Var'][insample_date].mean()*4
annulizedR_equal_insample=quarter_return['Equally'][insample_date].mean()*4
annulizedR_minv_insample=quarter_return['Min-Var'][insample_date].mean()*4
annulizedS_spx_insample=quarter_return['SPX'][insample_date].std()*2
annulizedS_meanv_insample=quarter_return['Mean-Var'][insample_date].std()*2
annulizedS_equal_insample=quarter_return['Equally'][insample_date].std()*2
annulizedS_minv_insample=quarter_return['Min-Var'][insample_date].std()*2
print("SPX",annulizedR_spx_insample,annulizedS_spx_insample)
print("Mean-Var",annulizedR_meanv_insample,annulizedS_meanv_insample)
print("Equally",annulizedR_equal_insample,annulizedS_equal_insample)
print("Min-Var",annulizedR_minv_insample,annulizedS_minv_insample)
sharpe_spx_insample=(annulizedR_spx_insample-risk_free)/annulizedS_spx_insample
sharpe_meanv_insample=(annulizedR_meanv_insample-risk_free)/annulizedS_meanv_insample
sharpe_equal_insample=(annulizedR_equal_insample-risk_free)/annulizedS_equal_insample
sharpe_minv_insample=(annulizedR_minv_insample-risk_free)/annulizedS_minv_insample
print("SPX Sharpe",sharpe_spx_insample)
print("Mean-Var Sharpe",sharpe_meanv_insample)
print("Equally Sharpe",sharpe_equal_insample)
print("Min-Var Sharpe",sharpe_minv_insample)

SPX 0.001531531093822985 0.21955274735546973
Mean-Var 0.2565791830967157 0.29155309241664823
Equally 0.26616405706243595 0.3011567507349648
Min-Var 0.26844700713719916 0.29823334992044453
SPX Sharpe -0.061345025595925314
Mean-Var Sharpe 0.828594137329483
Equally Sharpe 0.8339977651155983
Min-Var Sharpe 0.8498278519314074
